In [3]:
from build_dictionary import tokenizer
from get_data import GroupData, get_filename, get_data
from analysis_query import spelling_correction, AugmentedQuery
from get_simularity import Score
from dp_similarity import SentenceTransformers
from main import main
import pandas as pd
pd.set_option('display.width', 6000)

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import pickle

import nltk
# nltk.download('wordnet')
import numpy as np

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

# Outline
* Explain the components and functions of the search engine
* Demonstrate demos and test the speed of different cases
* Discuss some possible future extensions

# Components

## Data Loader - store each blog as objects

In [2]:
class GroupData:
    def __init__(self, blog_id, user_id, gender, age, industry, astrology, date, post):
        self.blog_id = blog_id
        self.user_id = user_id
        self.gender = gender
        self.age = age
        self.industry = industry
        self.astrology = astrology
        self.date = date
        self.post = post

In [3]:
with open('./group_data_objects.pickle', 'rb') as f:
    data_lists = pickle.load(f) # Read a pickle file

    
i = data_lists[40]
print("Blog ID:\t\t", i.blog_id, "\nUser ID:\t\t", i.user_id,
      "\nUser's Gender:\t\t", i.gender, "\nUser's Age:\t\t", i.age,
      "\nUser's Industry:\t", i.industry, "\nUser's Astrology:\t", i.astrology,
      "\nPosting Date:\t\t", i.date, "\nPosted blog:\t\t", i.post, '\n')

Blog ID:		 40 
User ID:		 3489929 
User's Gender:		 female 
User's Age:		 25 
User's Industry:	 Student 
User's Astrology:	 Cancer 
Posting Date:		 23,July,2004 
Posted blog:		 urlLink        Why this is, I do not know. But, OK. 



## Vocaburary Dictionaries

#### 1) A Tokenizer - split paragraphs into a list with all the single tokens

In [4]:
blog = ["Why this is, I do not know. But, OK."]
tokenized_blog = tokenizer(blog)[0]  # spaCy
print("Tonkenized blog:\n", tokenized_blog)

Tonkenized blog:
 ['why', 'this', 'be', ',', 'i', 'do', 'not', 'know', '.', 'but', ',', 'ok', '.']


#### 2) Four Dictionaries

   * Vocaburary Dictionary:  {word &rarr; word frequency in the whole dataset}

   * Word-to-ID & ID-to-Word: {word &rarr; word's unique ID; wordword's unique ID &rarr; word}

   * Posting Lists: {word's upique ID &rarr; (blog ID, word frequence in this blog, Word frequency in the whole dataset)}

In [5]:
voc_dic = pickle.load(open('voc_dic.pickle', 'rb'))
print("5 samples of vocaburary dictionary:\n", list(voc_dic.items())[:5], "\n")
voc2id = pickle.load(open('voc2id.pickle', 'rb'))
print("5 samples of word to id:\n", list(voc2id.items())[1:6], "\n")
id2voc = pickle.load(open('id2voc.pickle', 'rb'))
print("5 samples of id to word:\n", list(id2voc.items())[:5], "\n")
posting_list = pickle.load(open('posting_list.pickle', 'rb'))
first_posting_list = next(iter(posting_list.items()))
print("5 samples of posting list:\n", first_posting_list[0], first_posting_list[1][0:5],"\n")

5 samples of vocaburary dictionary:
 [('destiny', 1803), ('...', 1001878), ('hear', 36743), ('chosen', 3411), ('life', 167720)] 

5 samples of word to id:
 [('destiny', 0), ('...', 1), ('hear', 2), ('chosen', 3), ('life', 4)] 

5 samples of id to word:
 [(0, 'destiny'), (1, '...'), (2, 'hear'), (3, 'chosen'), (4, 'life')] 

5 samples of posting list:
 0 [(0, 3, 169), (1, 1, 183), (213, 1, 866), (373, 1, 201), (591, 1, 247)] 



## Query Pre-processors

#### 1) A Tokenizer
  * Recognize name entity

In [6]:
query = ["who is not happy in New York"]
tokenized_query = tokenizer(query)[0]  # spaCy
print(tokenized_query)

['who', 'be', 'not', 'happy', 'in', 'new york']


#### 2) Spelling Correction (demonstrated in the demo)

#### 3) Augmentation

  * Motivation
       * even if some blogs do not contain the excatly keywords in the query,  the system can still consider them to be relevant based on the added augmentation data

  * Presupposition
       * even if a blog does not contain the excatly keywords in the query, if it contains many semantically related words to the keywords, we can assume that the blog is probably relevant in some way

  * Steps 1
    * if negation marker "not" or "n't" appears, get the antonyms of the token being modified
    * remove the phrase including "not" (eg. "not happy")
    * add antonyms to the query token list

In [6]:
query = ["who is not happy in New York"]
tokenized_query = tokenizer(query)[0]  # spaCy
augm_q = AugmentedQuery(tokenized_query)
augment_obj = augm_q.augment_query()

In [7]:
print("Original Query:\n", tokenized_query)
print("Antonyms of 'happy':\n", augment_obj.antonyms_set)
for token in augment_obj.delete_set:
   tokenized_query.remove(token)
print("Deleted 'not' phrase:\n", tokenized_query)
tokenized_query += list(augment_obj.antonyms_set)
print("Revised Query:\n", tokenized_query)

Original Query:
 ['who', 'be', 'not', 'happy', 'in', 'new york']
Antonyms of 'happy':
 {'unhappy'}
Deleted 'not' phrase:
 ['who', 'be', 'in', 'new york']
Revised Query:
 ['who', 'be', 'in', 'new york', 'unhappy']


  * Steps 2
     * Get each token(except stopwords)'s synonyms, definition of synonyms, hyponyms, hypernyms
     * Put them in a set as related words

In [9]:
print("Synonyms:", augment_obj.synonyms_set, '\n')
print("Definition of Synonyms:", augment_obj.definition_set, '\n')
print("Hyponyms:", augment_obj.hyponyms_set, '\n')
print("Hypernyms:", augment_obj.hypernyms_set, '\n')
related_words = list(augment_obj.synonyms_set|augment_obj.definition_set|augment_obj.hyponyms_set|augment_obj.hypernyms_set)
print("Related Words:", related_words)

Synonyms: {'New York State', 'Empire State', 'New York City', 'Greater New York', 'New York', 'NY'} 

Definition of Synonyms: {'city', 'locate', 'colony', 'new', '13', 'southeastern', 'york', 'original', 'mouth', 'state', 'form', 'major', 'mid', 'large', 'river', 'center', 'cultural', 'hudson', 'financial', 'british', 'united'} 

Hyponyms: set() 

Hypernyms: set() 

Related Words: ['city', 'locate', 'New York City', 'colony', 'new', '13', 'southeastern', 'york', 'Empire State', 'NY', 'original', 'New York State', 'mouth', 'state', 'form', 'Greater New York', 'New York', 'major', 'mid', 'large', 'river', 'cultural', 'hudson', 'financial', 'center', 'british', 'united']


  * another example to show what is hypernyms and hyponyms

<img style="float: center;" src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/b4/Hyponym_and_hypernym.svg/1200px-Hyponym_and_hypernym.svg.png" width="70%"> 

In [10]:
query = ['purple']
augmented_q = AugmentedQuery(query)
augment_obj = augmented_q.augment_query()
print("Hypernyms: ", '\n', augment_obj.hypernyms_set, '\n')
print("Hyponyms: ", '\n', augment_obj.hyponyms_set)

Hypernyms:  
 {'nobility', 'color', 'colorise', 'discolor', 'colour in', 'colourise', 'noblesse', 'discolour', 'colorize', 'chromatic colour', 'colourize', 'color in', 'colour', 'spectral color', 'chromatic color', 'spectral colour'} 

Hyponyms:  
 {'lavender', 'reddish purple', 'violet', 'mauve', 'royal purple', 'reddish blue'}


## Computing Similarity

#### 1) TF-IDF (first rank)
  * Score each blog that contains at least one target word 
    * create a dictionary for each query
    * take the blog ID containing the target word as the keyword
    * add up the tf-idf scores for each contained target word as the value
    * rank the blogs by score

In [11]:
print("Revised Query:", tokenized_query)
print("Related Words:", related_words)

Revised Query: ['who', 'be', 'in', 'new york', 'unhappy']
Related Words: ['city', 'locate', 'New York City', 'colony', 'new', '13', 'southeastern', 'york', 'Empire State', 'NY', 'original', 'New York State', 'mouth', 'state', 'form', 'Greater New York', 'New York', 'major', 'mid', 'large', 'river', 'cultural', 'hudson', 'financial', 'center', 'british', 'united']


* The words in the 'Revised Query' and 'Related Words' are both considered as target word
* The TF-IDF score for each 'Related Words' was multiplied by 0.2, as it was considered less important than the query word

#### 2) Embeddings (re-rank)
  * Motivation
      * the ranking results according to tf-idf scores are influenced by the scope of the blog content covered
      * after testing, the high scoring results include exam papers and questionnaires
  * Presupposition
      * using a pre-trained language model to obtain sentence embeddings and calculating cosine similarity scores for queries and each blog may help to obtain semantically more relevant results
  * Pre-trained language model: MiniLM

# Demos

## Regular Search

In [2]:
main() # New York

Query example:

 #1 New York 	(Sensitive to named entities)
 #2 I'm not happy 	(Understand adjective phrases modified by 'not')
 #3 apple slow 	(Ambiguity)
 #4 apple pie 	(Ambiguity)
 #5 Americam 	(Auto correct typo)

What do you search for:
New York

Loading...
Ranking...
Ready to return results...
Running time:  18.23 s

       Score                                               Post              Date  Blog ID  User ID  Gender Age              Industry    Astrology
0   0.806208                                    before new york    01,August,2004   354351  1499735  female  24                indUnk       Pisces
1   0.699530                                 New York   urlLink      08,July,2004   208066  3586075    male  46                indUnk        Virgo
2   0.645598  New York as told by ME   Here's the  urlLink p...    04,August,2004    64234   946952  female  25             Education          Leo
3   0.617351  This is the weather page for the part of New Y...     09,March,2004   394

## Ambiguous Words

In [3]:
main() # apple slow

Query example:

 #1 New York 	(Sensitive to named entities)
 #2 I'm not happy 	(Understand adjective phrases modified by 'not')
 #3 apple slow 	(Ambiguity)
 #4 apple pie 	(Ambiguity)
 #5 Americam 	(Auto correct typo)

What do you search for:
apple slow

Loading...
Ranking...
Ready to return results...
Running time:  19.453 s

       Score                                               Post               Date  Blog ID  User ID  Gender Age    Industry    Astrology
0   0.598414                                       mmm...apple.  08,September,2003   619870   955372  female  16     Student        Aries
1   0.548954  urlLink Technology Special Report: The Future ...    24,January,2002   549643   320317    male  36  Technology       Pisces
2   0.524693  urlLink ZDNet: Story: Will Microsoft pull the ...    28,January,2002   549634   320317    male  36  Technology       Pisces
3   0.524136  Apple may come out with a new display.  This p...      17,March,2004   146994  2389810    male  23     Stu

In [6]:
main() # apple pie

Query example:

 #1 New York 	(Sensitive to named entities)
 #2 I'm not happy 	(Understand adjective phrases modified by 'not')
 #3 apple slow 	(Ambiguity)
 #4 apple pie 	(Ambiguity)
 #5 Americam 	(Auto correct typo)

What do you search for:
apple pie

Loading...
Ranking...
Ready to return results...
Running time:  16.009 s

       Score                                               Post               Date  Blog ID  User ID  Gender Age      Industry    Astrology
0   0.769318                                               pie.       22,June,2004    49573  3716795    male  16        indUnk  Sagittarius
1   0.695093  pie, as in, the actual, sugary, tasty, pastry ...    28,October,2003   149578   925742    male  16       Student       Gemini
2   0.640217                                           heha pie   25,February,2004   607885  2543884    male  13        indUnk       Pisces
3   0.640217                                           heha pie   25,February,2004   219908  3762737  female  13 

## Typo

In [5]:
main() # Americam

Query example:

 #1 New York 	(Sensitive to named entities)
 #2 I'm not happy 	(Understand adjective phrases modified by 'not')
 #3 apple slow 	(Ambiguity)
 #4 apple pie 	(Ambiguity)
 #5 Americam 	(Auto correct typo)

What do you search for:
Americam

Loading...
Do you mean "american"?
Yes, No, or Additional options required? [y/n/a]
a
{0: ['americas'], 1: ['american'], 2: ['america']}
choose one candidates or keep the original? [number/o]
2
Ranking...
Ready to return results...
Running time:  20.424 s

       Score                                               Post            Date  Blog ID  User ID  Gender Age              Industry    Astrology
0   0.635265                  The world according to America...  09,August,2004   415066   894945    male  27            Technology       Cancer
1   0.635265                  The world according to America...    28,June,2004   101595   705633    male  27            Technology       Gemini
2   0.630302                                 God Bless A

In [10]:
tokenized_query = ['america']
augm_q = AugmentedQuery(tokenized_query)
augment_obj = augm_q.augment_query()
print("Synonyms:", augment_obj.synonyms_set, '\n')
print("Definition of Synonyms:", augment_obj.definition_set, '\n')
print("Hyponyms:", augment_obj.hyponyms_set, '\n')
print("Hypernyms:", augment_obj.hypernyms_set, '\n')

Synonyms: {'US', 'America', 'United States', 'United States of America', 'USA', 'the States', 'U.S.', 'U.S.A.'} 

Definition of Synonyms: {'independence', 'island', 'central', '1776', 'northwest', 'north', 'plus', 'achieve', 'south', '50', 'america', 'state', 'conterminous', 'contain', 'alaska', '-', 'pacific', 'american', 'ocean', 'republic', 'hawaiian', '48'} 

Hyponyms: set() 

Hypernyms: set() 



# Extentions

## Knowledge Graphs


### Disambiguate
  * e.g. DBpedia dbo:wikiPageDisambiguates

### Search for semantically relevant concepts and entities

In [5]:
from normalize_uris import normalize_uris
from query_dbpedia import query

def describe_resource(in_data):
    uri = normalize_uris(in_data)
    q1 = "PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> SELECT ?types WHERE { <" + uri + "> rdf:type ?types}"
    results1 = query(q1)

    types = 'all the types: \n'
    for result in results1["results"]["bindings"]:
        types += result["types"]["value"] + '\n'

    q2 = "SELECT DISTINCT ?property WHERE { <" + uri + "> ?property ?value .}"
    # FILTER (!isLiteral(?property))
    #q2 = "SELECT DISTINCT ?property WHERE { <" + uri + "> ?property ?value .}"
    results2 = query(q2)

    distinct = "all the distinct properties: \n"
    for result in results2["results"]["bindings"]:
        distinct += result["property"]["value"] + '\n'

    return types + distinct

In [12]:
uri_data_1 = 'dbr:New_York_City'
print(describe_resource(uri_data_1))

all the types: 
http://www.w3.org/2002/07/owl#Thing
http://dbpedia.org/ontology/Place
http://dbpedia.org/ontology/Location
http://www.w3.org/2003/01/geo/wgs84_pos#SpatialThing
http://dbpedia.org/class/yago/Object100002684
http://dbpedia.org/class/yago/Organization108008335
http://dbpedia.org/class/yago/PhysicalEntity100001930
http://dbpedia.org/class/yago/Region108630985
http://dbpedia.org/class/yago/School108276720
http://dbpedia.org/class/yago/Seat108647945
http://dbpedia.org/class/yago/SecondarySchool108284481
http://dbpedia.org/class/yago/Site108651247
http://dbpedia.org/class/yago/SocialGroup107950920
http://dbpedia.org/class/yago/StateCapital108695539
http://dbpedia.org/class/yago/Tract108673395
http://dbpedia.org/class/yago/UrbanArea108675967
http://dbpedia.org/class/yago/WikicatPopulatedCoastalPlacesInNewYork
http://dbpedia.org/class/yago/WikicatPopulatedPlacesEstablishedIn1624
http://dbpedia.org/class/yago/WikicatPopulatedPlacesOnTheHudsonRiver
http://dbpedia.org/class/yago/Wi

In [8]:
uri_data_2 = 'http://dbpedia.org/resource/Germany'
print(describe_resource(uri_data_2))

all the types: 
http://www.w3.org/2002/07/owl#Thing
http://dbpedia.org/ontology/Place
http://dbpedia.org/ontology/Location
http://www.w3.org/2003/01/geo/wgs84_pos#SpatialThing
http://dbpedia.org/class/yago/Object100002684
http://dbpedia.org/class/yago/PhysicalEntity100001930
http://dbpedia.org/class/yago/Region108630985
http://dbpedia.org/class/yago/YagoGeoEntity
http://dbpedia.org/class/yago/YagoLegalActorGeo
http://dbpedia.org/class/yago/YagoPermanentlyLocatedEntity
http://dbpedia.org/class/yago/WikicatMemberStatesOfNATO
http://dbpedia.org/class/yago/WikicatMemberStatesOfTheEuropeanUnion
http://dbpedia.org/class/yago/WikicatMemberStatesOfTheUnionForTheMediterranean
http://dbpedia.org/class/yago/WikicatMemberStatesOfTheUnitedNations
http://dbpedia.org/class/yago/WikicatStatesAndTerritoriesEstablishedIn1871
http://dbpedia.org/class/yago/WikicatStatesAndTerritoriesEstablishedIn1949
http://schema.org/Place
http://dbpedia.org/class/yago/WikicatCountries
http://dbpedia.org/class/yago/Wikic